In [2]:
df = pd.read_csv('CleanedDataNew.csv')

In [1]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import pos_tag
from string import punctuation,digits
import re
from keras.utils import to_categorical
#from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
#from pandas_ml import ConfusionMatrix
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Bidirectional
from keras.layers import Embedding
# from keras.utils.np_utils import probas_to_classes
from sklearn import preprocessing
from keras.models import Sequential
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
from sklearn.model_selection import train_test_split
%matplotlib inline
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from matplotlib import rcParams
from random import shuffle
# figure size in inches
rcParams['figure.figsize'] = 11.7,8.27
#import word2vecReader as godin_embedding
#from gensim.models import KeyedVectors
from sklearn.model_selection import StratifiedKFold
#import skopt
#from skopt import gp_minimize, forest_minimize
#from skopt.space import Real, Categorical, Integer
#from skopt.utils import use_named_args


C:\Users\Anubha\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df.head()
df_new= df.drop(columns=['url'])

In [4]:
df_new.head()


,text,target
0,Century-old mud houses amidst concrete jungle:...,0
1,Group of ministers formed to unlock land of cl...,0
2,Dacoits batter infant in Garia: Seven armed da...,1
3,TMC warns workers against sand mining: A day a...,0
4,'Policymakers should have bigger vision': A se...,0


In [5]:
len(df_new['text'])

6326

In [5]:
def remove_punctuation(s):
    list_punctuation = list(punctuation)
    for i in list_punctuation:
        s = s.replace(i,' ')
    return s

In [6]:
def clean_sentence(sentence):
    sentence = sentence.lower()
    #remove multiple repeat non num-aplha char !!!!!!!!!-->!
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
#     print(sentence)
    #removes alpha char repeating more than twice aaaa->aa
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
#     print(sentence)
    #removes links
    #sentence = re.sub(r'(?P<url>https?://[^\s]+)', r'', sentence)
#     print(sentence)
    # remove @usernames
    sentence = re.sub(r"\@(\w+)", "", sentence)
#     print(sentence)
    #removing stock names to see if it helps
#     sentence = re.sub(r"(?:\$|https?\://)\S+", "", sentence)
    #remove # from #tags
    sentence = sentence.replace('#','')
    sentence = sentence.replace("'s",'')
    sentence = sentence.replace("-",' ')
#     print(sentence)
    # split into tokens by white space
    tokens = sentence.split()
    # remove punctuation from each token
    tokens = [remove_punctuation(w) for w in tokens]
#     print(tokens)
    #     remove remaining tokens that are not alphabetic
#     tokens = [word for word in tokens if word.isalpha()]
#no removing non alpha words to keep stock names($ZSL)
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
#     for w in stop_words:
#         print(w)
#     print(tokens)
    # filter out short tokens
#     tokens = [word for word in tokens if len(word) > 1]
#     print(tokens)
    remove_digits = str.maketrans('', '', digits)
#     print(tokens)
    tokens = [w.translate(remove_digits) for w in tokens]
    tokens = [w.strip() for w in tokens]
    tokens = [w for w in tokens if w!=""]
#     print(tokens)
    tokens = ' '.join(tokens)
    return tokens

In [7]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [8]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [9]:
#not needed here since this is binary classification 
def convert_lables(trainY,testY):
    le = preprocessing.LabelEncoder()
    le.fit(trainY+testY)
    temp1 = le.transform(trainY)
    return to_categorical(temp1,no_of_classes),le.classes_

In [2]:
#loading GloVe embedding
def load_GloVe_embedding(file_name):
    embeddings_index = dict()
    f = open(file_name,encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index

In [3]:
# create a weight matrix for words in training docs
def get_GloVe_embedding_matrix(embeddings_index):
    embedding_matrix = np.zeros((vocab_size, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [12]:
sentence= df_new['text']
labels  = df_new['target']

In [13]:
sentence = [clean_sentence(x) for x in sentence]

In [14]:
lengths = [len(s.split()) for s in sentence]

In [16]:
print('max len = ',max(lengths))
#sns.distplot(lengths)


max len =  20


In [17]:
#Deviding it into train and test
max_length  = max(lengths)
X_train = sentence[:4000]
X_test = sentence[4000:]
Y_train= labels[:4000]
Y_test = labels[4000:]

In [19]:
#Training data
tokenizer = create_tokenizer(X_train)
vocab_size = len(tokenizer.word_index) + 1
trainX = encode_text(tokenizer, X_train, max_length)
# testX = encode_text(tokenizer, testX, max_length)
#trainY,lable_encoding = convert_lables(Y,[])
#Testing data
tokenizer = create_tokenizer(X_test)
vocab_size = len(tokenizer.word_index) + 1
testX = encode_text(tokenizer, X_test, max_length)

In [20]:
print(vocab_size)
print(len(trainX))

print(len(X_train))

11313
4000
4000


In [1]:
embeddings_index_glove = load_GloVe_embedding('glove.6B.100d.txt')

NameError: name 'load_GloVe_embedding' is not defined

In [22]:
len(embeddings_index_glove)

400000

In [23]:
get_embeddings = get_GloVe_embedding_matrix(embeddings_index_glove)

In [62]:
#For each sentence
#1. See all words's emebeddings (of size 100 for each word)
#2. Take the mean of A[0] ,B[0], C[0].....N[0], such that A, B,C .. N are words of embeddings.
#3. Do this till A[100]...N[100]
#4. This creates a vector sentence1 : [mean of first bit of  words........... mean of last bit of words]


In [81]:
#Finds the mean of all the words in the sentence
def mean_vector_sentence(sentence):
    l1=[]
    for word in sentence.split():
        l1.append(get_embeddings[tokenizer.word_index[word]])
    mat = np.array(l1)
    v=np.mean(mat)
    return v
        
    

In [82]:
#Each sentence is now a vector , taken by mean of vectors of the words in the sentence
vect_sentence=[]
for s in sentence:
    vect_sentence.append(mean_vector_sentence(s))

In [85]:
len(vect_sentence)
len(Y)

6326

In [86]:
#Now for example title for each title will be created such.
#Will content embeddings be done sentence wise or the entire row content into  one vector
#These embeddings will be further used in the cnn network for the fused main model.

In [27]:
embedding_dim = 256
filter_sizes =4
num_filters = 512
drop = 0.4

epochs = 100
batch_size = 10

In [ ]:
from PIL import Image as PImage
from os import listdir
from keras.models import Model
from keras.applications.vgg19 import decode_predictions
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19
#function to find the image vector fro pre-trained VGG19 Model
#Datasets of fake and real images for each article
#file being the name of the folder of images.
def image_embeddings(file):
    imagesList = listdir(file)
    


    model= VGG19(weights='imagenet')
    
    final_image = []
    for img in imagesList:
        img= file+'/'+img
        #img = PImage.open(file +'/'+ img)
        image = load_img(img, target_size=(224, 224))
# convert the image pixels to a numpy array
        image = img_to_array(image)
# reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
        image = preprocess_input(image)
# predict the probability across all output classes
        yhat = model.predict(image)
        final_image.append(yhat)
# convert the probabilities to class labels
        label = decode_predictions(yhat)
        
# retrieve the most likely result, e.g. highest probability
        label = label[0][0]
# print the classification
       # print('%s (%.2f%%)' % (label[1], label[2]*100))
    return final_image
        

In [ ]:
# define the model
def define_model(length_title, vocab_size_title, length_content , vocab_size_content,rel_embeddings):
	# channel 1 : title
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size,100)(inputs1)
    #Filter size:4 
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
	pool1 = MaxPooling1D(pool_size=2)(conv1)
	flat1 = Flatten()(pool1)
    
    
	# channel 2 :  CONTENT + KG
	inputs2 = Input(shape=(length_content,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding2)
	#drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
    
	#should this be embedding content kalength?
	inputs3 = Input(shape=(length_content,))
	#embedding3 = Embedding(vocab_size, 100)(inputs3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(rel_embedding)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
    
    merged_content = concatenate([flat2,flat3])
   
    #IMAGES
    inputs4 = #max length of  of image embedding
    image_embedding = image_embeddings('fake')
    
	# merge
	merged = concatenate([flat1, merged_content ,image_embeddings])
    #merged2= conactenate(image_embeddings)
	# interpretation
    
	dense1 = Dense(10, activation='relu')(merged)
    #dense2 = Dense(10,activation='relu')(merged2)
	outputs = Dense(1, activation='softmax')(dense1)
    #what should be put in input for  image embedding?
	model = Model(inputs=[inputs1, inputs2, inputs3, inputs4], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='final.png')
	return model

In [24]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam

In [25]:
#Model for title
def define_model(length, vocab_size):
	# channel 1 : title
	inputs1 = Input(shape=(length,))
    #embedding dimention :256
	embedding1 = Embedding(vocab_size,100)(inputs1)
    #Filter size:4 
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    #This is what I think it should be,becuase embedding dimention is256
    #conv2= Conv2D(num_filters, kernel_size=(4,256), padding='valid', kernel_initializer='normal', activation='relu')(embedding1)
	#drop1 = Dropout(0.5)(conv1)
    #pool1 = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(conv2)
	pool1 = MaxPooling1D(pool_size=2)(conv1)
	flat1 = Flatten()(pool1)
	merged = flat1
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=inputs1, outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	print(model.summary())
	return model
    
    

In [26]:
model =define_model(max_length,vocab_size)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 100)           1131300   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 17, 32)            12832     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 32)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total para

In [27]:
history_object = model.fit(trainX,Y_train,epochs=10, batch_size=64)


Epoch 1/10
4000/4000 [==============================] - 4s 892us/step - loss: 0.3593 - acc: 0.9170
Epoch 2/10
4000/4000 [==============================] - 2s 432us/step - loss: 0.1784 - acc: 0.9245
Epoch 3/10
4000/4000 [==============================] - 2s 435us/step - loss: 0.0749 - acc: 0.9245
Epoch 4/10
4000/4000 [==============================] - 2s 440us/step - loss: 0.0561 - acc: 0.9708
Epoch 5/10
4000/4000 [==============================] - 2s 430us/step - loss: 0.0496 - acc: 0.9988
Epoch 6/10
4000/4000 [==============================] - 2s 432us/step - loss: 0.0215 - acc: 0.9995
Epoch 7/10
4000/4000 [==============================] - 2s 442us/step - loss: 0.0046 - acc: 0.9995
Epoch 8/10
4000/4000 [==============================] - 2s 432us/step - loss: 0.0024 - acc: 0.9998
Epoch 9/10
4000/4000 [==============================] - 2s 452us/step - loss: 0.0019 - acc: 0.9998
Epoch 10/10
4000/4000 [==============================] - 2s 424us/step - loss: 3.8362e-04 - acc: 1.0000


In [29]:
p=model.predict(testX)

In [30]:
p #Will this go into the model in final predict? #Ask Simra

array([[1.15608265e-04],
       [1.26477360e-04],
       [8.09503399e-06],
       ...,
       [1.51686225e-04],
       [5.76546190e-05],
       [3.76895279e-08]], dtype=float32)

In [31]:
len(p)

2326

In [34]:
model.evaluate(testX,Y_test)

2326/2326 [==============================] - 0s 75us/step


[0.5685145368780348, 0.9333619946871763]